In [55]:
import pandas as pd
pd.options.display.max_columns=99
pd.options.display.max_rows=99
import numpy as np

%matplotlib inline

In [56]:
path="../data/"
cvr_file=path+'CVR_LOBBY_DISCLOSURE_CD.TSV'
cvr_array=[]
with open (cvr_file) as f:
    l=0
    for line in f:
        row=((line.split('\t')))
        if len(row)==52:
            cvr_array.append(row)
            l+=1
columns=cvr_array[0]
cvr_array.pop(0)
cvr=pd.DataFrame(cvr_array,columns=columns)
cvr.shape

(301286, 52)

In [57]:
pay_file=path+'LPAY_CD.TSV'
pay_array=[]
with open (pay_file) as f:
    l=0
    for line in f:
        row=((line.split('\t')))
        if len(row)==26:
            pay_array.append(row)
            l+=1
columns=pay_array[0]
pay_array.pop(0)
pay=pd.DataFrame(pay_array,columns=columns)
pay.shape

(466026, 26)

In [85]:
pay['FILING_ID']=pd.to_numeric(pay.FILING_ID)
pay['CUM_TOTAL']=pd.to_numeric(pay.CUM_TOTAL)
pay['PER_TOTAL']=pd.to_numeric(pay.PER_TOTAL)
dg=pay.groupby(['FILING_ID','AMEND_ID']).sum()
dg.reset_index(inplace=True)

In [84]:
cvr['FILING_ID']=pd.to_numeric(cvr.FILING_ID)

In [59]:
cvr['report_date']=pd.to_datetime(cvr.RPT_DATE,errors='coerce')

In [60]:
cvr['report_year']=cvr.report_date.dt.year

In [61]:
df=cvr[cvr.report_date.dt.year==2016]
df.shape

(23386, 54)

In [86]:
dm=df.merge(dg,how='left',on=['FILING_ID','AMEND_ID'])
dm.shape

(23386, 57)

In [63]:
dm=dm[~dm.PER_TOTAL.isnull()]
dm=dm[dm.PER_TOTAL>0]
dm.reset_index(inplace=True,drop=True)
dm.shape

(11336, 56)

In [64]:
dm_cut=dm[['FILING_ID','AMEND_ID','report_date','FILER_NAML','PER_TOTAL','CUM_TOTAL']]
dm_cut.shape

(11336, 6)

In [65]:
dm_cut.to_csv(path+"lobbying_reports.csv",index=False,date_format="%Y-%m-%d")
dm_cut.shape

(11336, 6)

In [93]:
dm_cut.sort_values('report_date',ascending=False)

,FILING_ID,AMEND_ID,report_date,FILER_NAML,PER_TOTAL,CUM_TOTAL
10122,2101782,0,2016-12-31,"Los Angeles Turf Club, Incorporated and its Af...",45000.00,315000.00
10861,2099030,1,2016-12-12,RIVERSIDE COUNTY OFFICE OF EDUCATION,1595.20,42361.04
10258,2101581,1,2016-12-12,"GILL RANCH STORAGE, LLC",15000.00,32500.00
10252,2110325,0,2016-12-10,WALKER STRATEGIES,34802.12,34802.12
11178,2094539,1,2016-12-09,COUNTY HEALTH EXECUTIVES ASSOCIATION OF CALIFO...,26327.59,145212.47
10251,2099180,1,2016-12-09,WILLIAM AND FLORA HEWLETT FOUNDATION; THE,2608.75,32325.27
11179,2060812,1,2016-12-09,UNIVERSITY OF SOUTHERN CALIFORNIA; THE,20000.00,90000.00
10250,2103274,1,2016-12-07,METROPOLITAN TRANSPORTATION COMMISSION,32332.50,108999.14
10248,2060967,1,2016-12-06,"ETAGEN, INC.",9750.00,59250.00
10249,2087623,1,2016-12-06,"WHEELS FINANCIAL GROUP, LLC D/B/A LOANMART",22500.00,150642.20
